## POS tagging using modified Viterbi

### Data Preparation

In [19]:
#Importing libraries
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [20]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:2])

In [24]:
# Splitting into train and test
train_set, test_set = train_test_split(nltk_data,test_size=0.05, random_state =100)
print(len(train_set))
print(len(test_set))

3718
196


In [25]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95949

In [26]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
print(tokens[:10])
# vocabulary
V = list(set(tokens))
print(len(V))

['One', 'bright', 'sign', 'is', 'that', 'a', 'growing', 'number', 'of', 'women']
12106


In [27]:
# number of tags
T = list(set([pair[1] for pair in train_tagged_words]))
print(len(T))
print(T)

12
['CONJ', 'PRT', 'DET', '.', 'NOUN', 'ADV', 'ADP', 'X', 'PRON', 'VERB', 'ADJ', 'NUM']


### Build the vanilla Viterbi based POS tagger

### Emission Probabilities and Transition Probabilities

In [29]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag):
    tag_list = [pair for pair in train_tagged_words if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag/count_tag)



# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1/count_t1)

### Tags Matrix

In [32]:
# creating t x t transition matrix of tags. Each column is t2, each row is t1. Thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)
        
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,CONJ,PRT,DET,.,NOUN,ADV,ADP,X,PRON,VERB,ADJ,NUM
CONJ,0.000464,0.003709,0.118683,0.035698,0.350487,0.053778,0.053778,0.008809,0.058414,0.155308,0.118683,0.042188
PRT,0.002297,0.001969,0.101050,0.043635,0.245735,0.010171,0.019357,0.013123,0.017717,0.405184,0.083661,0.056102
DET,0.000481,0.000240,0.005771,0.017913,0.637293,0.012623,0.009618,0.045323,0.003727,0.040394,0.204977,0.021640
.,0.058032,0.002511,0.173558,0.092923,0.222531,0.052292,0.092206,0.026908,0.065208,0.088708,0.043681,0.081353
NOUN,0.042921,0.043357,0.013363,0.239951,0.264280,0.016813,0.177058,0.028868,0.004721,0.146955,0.012165,0.009550
ADV,0.006991,0.014314,0.069907,0.135153,0.031624,0.077230,0.119507,0.023302,0.015646,0.344541,0.130160,0.031624
ADP,0.000848,0.001484,0.323969,0.039754,0.321213,0.013357,0.017492,0.034984,0.069119,0.008481,0.107389,0.061910
X,0.010316,0.184891,0.055229,0.162831,0.062371,0.025393,0.144898,0.074433,0.055705,0.204571,0.016505,0.002857
PRON,0.004981,0.012261,0.009195,0.040613,0.211494,0.034100,0.023372,0.092720,0.007663,0.484291,0.072031,0.007280
VERB,0.005186,0.031427,0.133292,0.034291,0.111386,0.082050,0.091184,0.218438,0.035916,0.168744,0.065640,0.022448


## Viterbi Algorithm

#### Create plain version of Viterbi Algorithm which can not correctly identify the unknown words

In [33]:
# Viterbi Heuristic
def Viterbi_vanila(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #emission_p = word_given_tag(words[key], tag)
            #word_tag_df
            emission_p = word_given_tag(words[key], tag)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [34]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

##### Accuracy of vanila Viterbi alogorithm

In [35]:
%%time
tagged_seq = Viterbi_vanila(test_tagged_words)
print(tagged_seq[:10])

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

[('Investors', 'NOUN'), ('took', 'VERB'), ('advantage', 'NOUN'), ('of', 'ADP'), ('Tuesday', 'NOUN'), ("'s", 'PRT'), ('stock', 'NOUN'), ('rally', 'NOUN'), ('*-1', 'X'), ('to', 'PRT')]
CPU times: user 9min 16s, sys: 3.23 s, total: 9min 20s
Wall time: 9min 30s


### Solve the problem of unknown words

#### Defining the rule based function which can return the tag based on the pattern defined

In [178]:
# specify patterns for tagging. example from the NLTK book

test= 'samsung'

patterns = [
    (r'.*ing$', 'VERB'),              
    (r'.*ed$', 'VERB'),               
    (r'.*es$', 'VERB'),               
    (r'.*ly$', 'ADV'),              
    (r'.*ble$', 'ADJ'),                
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), 
    (r'.*', 'NOUN')                    
]

regexp_tagger = nltk.RegexpTagger(patterns)

def rulebased_tagging(test_words):
    return regexp_tagger.tag([test_words])[0][1]

def rulebased_tagging1(test_words):
    return regexp_tagger.tag([test_words])

# rulebased_tagging(test)

'NOUN'


### Updated Viterbi Algorithms for different techniques

### Probabilistic with Viterbi

In [38]:
# Updated Viterbi Heuristic

def Viterbi_probabilistic(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #emission_p = word_given_tag(words[key], tag)
            #word_tag_df
            if(words[key] in V):
                emission_p = word_given_tag(words[key], tag)
                state_probability = emission_p * transition_p
            else:
                state_probability = transition_p
                                   
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

### Probabilistic with rule based

In [182]:
# Updated Viterbi Heuristic

def Viterbi_rulebased(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        state_probability = 0
        rule_tag = ""
        if word[key] in v:
            for tag in T:     
                if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            
            # compute emission and state probabilities           
                emission_p = word_given_tag(words[key], tag)
                state_probability = emission_p * transition_p
            p.append(state_probability)
        
        pmax = max(p)
        state_max = T[p.index(pmax)]
        if word[key] in V:
            state.append(state_max)        
        else:
            rule_tag=rulebased_tagging(words[key])
            state.append(rule_tag) 
                
    return list(zip(words, state))

In [185]:
def Viterbi_rulebased(words):
    state = []
    T = list(set([pair[1] for pair in train_tagged_words]))
    for key, word in enumerate(words):
    #initialise list of probability column for a given observation
        p = []
        state_probability = 0
        rule_tag = ""
        if words[key] in V:
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]
                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)
                state_probability = emission_p * transition_p
                p.append(state_probability)
            pmax = max(p)
            state_max = T[p.index(pmax)]
            state.append(state_max)
            
        else:
            rule_tag=rulebased_tagging(words[key])
            state.append(rule_tag)
    return list(zip(words, state))

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [40]:
%%time
tagged_seq1 = Viterbi_probabilistic(test_tagged_words)
# accuracy
check = [i for i, j in zip(tagged_seq1, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

0.9331499894224666
CPU times: user 8min 37s, sys: 1.56 s, total: 8min 39s
Wall time: 8min 42s


In [ ]:
%%time
tagged_seq2 = Viterbi_rulebased(test_tagged_words)
# accuracy
check = [i for i, j in zip(tagged_seq2, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

In [99]:
from nltk.corpus import brown
# nltk.download('brown')
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')

import requests
response = requests.get('https://cdn.upgrad.com/UpGrad/temp/9dca5f3b-53c3-47e1-86d5-5ec5dafad6f0/Test_sentences.txt')
data = response.text

from nltk.tokenize import sent_tokenize, word_tokenize
sentense = sent_tokenize(data)
word_sample= word_tokenize(data)


# calculating tags using nltk.pos_tags, tagset universal

tagged_data=nltk.pos_tag(word_sample,tagset='universal')
# tagged_data
sample_sent =list([pair[0] for pair in tagged_data])
sample_sent[:5]
tagged_data[:5]

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN')]

In [102]:
# %%time
tagged_seq3 = Viterbi_vanila(sample_sent)
print(tagged_seq3[:5])

# accuracy
check = [i for i, j in zip(tagged_seq3, tagged_data) if i == j] 
accuracy = len(check)/len(tagged_seq3)
print(accuracy)

[('Android', 'CONJ'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN')]
0.7624309392265194


In [103]:
tagged_seq4 = Viterbi_probabilistic(sample_sent)
print(tagged_seq4[:5])

# accuracy
check = [i for i, j in zip(tagged_seq4, tagged_data) if i == j] 
accuracy = len(check)/len(tagged_seq4)
print(accuracy)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN')]
0.8674033149171271


In [186]:
%%time
tagged_seq5 = Viterbi_rulebased(sample_sent)
print(tagged_seq5[:5])

# accuracy
check = [i for i, j in zip(tagged_seq5, tagged_data) if i == j] 
accuracy = len(check)/len(tagged_seq5)
print(accuracy)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN')]
0.9281767955801105
CPU times: user 16 s, sys: 98.5 ms, total: 16.1 s
Wall time: 16.2 s
